In [1]:
import torch

torch.__version__

'2.0.0+cu117'

In [2]:
torch.cuda.is_available()

True

In [4]:
pwd

'C:\\Users\\Suhyeon\\Desktop\\TIL\\capstone design 02'

In [6]:
# 1. torchvision을 사용하여 CIFAR10을 불러오고 정규화 하기

import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4 

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')



import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 학습용 이미지를 무작위로 가져오기
dataiter = iter(trainloader)
images, labels = next(dataiter)

# 이미지 보여주기
imshow(torchvision.utils.make_grid(images))

# 정답(label) 출력
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))




# 2. 합성곱 신경망 정의
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 첫번째 nn.Conv2d 의 2번째 인자와 두번째 nn.Conv2d 의 1번째 인자는 같은 숫자여야 함
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # 배치를 제외한 모든 차원을 평탄화(flatten)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = ResNet()




# 3. 손실 함수 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)



## CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력합니다:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

net.to(device)



# 4. 학습용 데이터를 사용하여 신경망 학습
for epoch in range(2):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
    #    inputs, labels = data # cuda 사용
        inputs, labels = data[0].to(device), data[1].to(device)

        # 변화도(Gradient) 매개변수를 0으로 만들고
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


# 학습한 모델 저장
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)




# 5. 시험용 데이터로 신경망 검사하기
dataiter = iter(testloader)
images, labels = next(dataiter)

# 이미지를 출력합니다.
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))




# 저장했던 모델 불러오기
net = Net()
net.load_state_dict(torch.load(PATH))
outputs = net(images)

# 가장 높은 값을 갖는 인덱스(index)를 뽑기
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))




# 전체 데이터 셋에 대해 확인
correct = 0
total = 0
# 학습 중이 아니므로, 출력에 대한 변화도를 계산할 필요가 없습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # 신경망에 이미지를 통과시켜 출력을 계산합니다
        outputs = net(images)
        # 가장 높은 값(energy)를 갖는 분류(class)를 정답으로 선택하겠습니다
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')




# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')




100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


ModuleNotFoundError: No module named 'matplotlib'

In [7]:
conda install matplotlib

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Suhyeon\anaconda3\envs\torch_cuda11.6

  added / updated specs:
    - matplotlib


The following NEW packages will be INSTALLED:

  blas               pkgs/main/win-64::blas-1.0-mkl 
  brotli             pkgs/main/win-64::brotli-1.0.9-h2bbff1b_7 
  brotli-bin         pkgs/main/win-64::brotli-bin-1.0.9-h2bbff1b_7 
  contourpy          pkgs/main/win-64::contourpy-1.0.5-py39h59b6b97_0 
  cycler             pkgs/main/noarch::cycler-0.11.0-pyhd3eb1b0_0 
  fonttools          pkgs/main/noarch::fonttools-4.25.0-pyhd3eb1b0_0 
  freetype           pkgs/main/win-64::freetype-2.12.1-ha860e81_0 
  giflib             pkgs/main/win-64::giflib-5.2.1-h8cc25b3_3 
  glib               pkgs/main/win-64::glib-2.69.1-h5dc1a3c_2 
  gst-plugins-base   pkgs/main/win-64::gst-plugins-base-1.18.5-h9e645db_0 
  gstreamer          pkgs/main/win-64::gstreamer-1.18.5-hd78058f_0 
  icu                pkgs/main/win-64::icu-58.